In [ ]:
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

## Classifcation
  - A classification model is one which attempts to predict a class, or category. That is, predicting from a number of discrete possibilities such as 'dog' or 'cat'

## Regression
 - A regression model is one which attempts to predict one or more numberic quantities, such as temperature, or a location.

## Transfer learning
- Using a ***pretrained*** model for a task different to what it was originally trained for.
- more quickly, less data 


## The state of deep learning
- Vision
    - Detection, Classification
- Text
    - Classification, Conversation
- Tabular
    - High cardinality, GPU(Rapids)
- Recsys
    - Prediction(Recommendation)
- Multi-modal
    - Labeling, Captioning, Human in the loop
- Other
    - NLP -> Protein

# High Temperature and High Humidity Reduces the Transmission of COVID-19 
- How might we dicide if there's a relationship? 
  1. Pick a null hypothesis
    - e.g ***No relationship***
  2. Gather data of independent * dependent variables
    - Temperature & R ( R is a measureof transmissibility)
  3. What % of time would we see that relationship by chance ?

In [ ]:
key = 'API_KEY'

In [ ]:
search_images_bing

In [ ]:
results = search_images_bing(key, 'grizzly bear')
ims = results.attrgot('contentUrl')
len(ims)

In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
      print(o)
      dest = (path/o)
      dest.mkdir(exist_ok=True)
      results = search_images_bing(key, f'{o} bear')
      download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed= verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink)

# From data to DataLoaders

In [ ]:
bears = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct= 0.2, seed= 42),
    get_y = parent_label,
    item_tfms = Resize(128)
)

In [ ]:
dls = bears.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n= 4, nrows= 1)

# Training your model, and using it to clean your data.

In [ ]:
bears = bears.new(
    item_tfms= RandomResizedCrop(224, min_scale= 0.5),
    batch_tfms = aug_transforms()
)

dls = bears.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics= error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrow= 1)

# Tuning your model into an online application

## Using the model for inference

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts= '.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')